# Types and type declarations

In [1]:
x = 100.0
typeof(x)

Float64

In [2]:
function foo()
    x::Int8 = 100
    x
end

x = foo()
typeof(x)

Int8

## Type trees

In [3]:
using AbstractTrees
AbstractTrees.children(x::Type) = subtypes(x)

LoadError: ArgumentError: Package AbstractTrees not found in current path:
- Run `import Pkg; Pkg.add("AbstractTrees")` to install the AbstractTrees package.


In [4]:
print_tree(Any)

LoadError: UndefVarError: print_tree not defined

In [5]:
print_tree(AbstractArray)

LoadError: UndefVarError: print_tree not defined

## Types in function arguments

In [6]:
function g(x, y)
    return x * y
end

x = g(2, 3)
@show x
typeof(x)

x = 6


Int64

In [7]:
x = g(2.0, 3.0)
@show x
typeof(x)

x = 6.0


Float64

In [8]:
x = g("foo", "bar")
@show x
typeof(x)

x = "foobar"


String

In [9]:
function f(x::Number, y::Number)
    return x * y
end

x = f(2, 3)
@show x
typeof(x)

x = 6


Int64

In [10]:
x = f("foo", "bar")

LoadError: MethodError: no method matching f(::String, ::String)

## Type Unions

In [11]:
function h(x::Union{Integer, String}, y::Union{Integer, String})
    x * y
end

@show h(1, 2)
@show h("foo", "bar");

h(1, 2) = 2
h("foo", "bar") = "foobar"


In [12]:
h(1.0, 2.0)

LoadError: MethodError: no method matching h(::Float64, ::Float64)

## Composite Types

In [13]:
struct Point2D
    x::Float64
    y::Float64
end

p = Point2D(1.0, 2.0)
@show p.x
@show p.y;

p.x = 1.0
p.y = 2.0


### Parametric Composite Types

In [14]:
struct Point3D{T}
    x::T
    y::T
    z::T
end

p = Point3D{Int8}(1, 2, 3)
@show p.x
typeof(p.y)

p.x = 1


Int8

In [15]:
p = Point3D{Float32}(1.0, 2.0, 3.0)
typeof(p.x)

Float32

In [16]:
p = Point3D{String}("foo", "bar", "foobar")
typeof(p.x)

String

### Type restrictions on parameters

In [17]:
struct NumericPoint2D{T<:Real}
    x::T
    y::T
end 

p = NumericPoint2D{Float64}(1.0, 2.0)
@show typeof(p.x)

q = NumericPoint2D{Int8}(1, 2)
@show typeof(q.x)

typeof(p.x) = Float64
typeof(q.x) = Int8


Int8

In [18]:
NumericPoint2D{String}("foo", "bar")

LoadError: TypeError: in NumericPoint2D, in T, expected T<:Real, got Type{String}

In [19]:
import Base.+

### Functions and operators with parametric types

In [20]:
+(A::NumericPoint2D{T}, B::NumericPoint2D{T}) where T = NumericPoint2D{T}(A.x + B.x, A.y + B.y)

+ (generic function with 209 methods)

In [21]:
A = NumericPoint2D{Int32}(1, 2)
B = NumericPoint2D{Int32}(1, 2)

A + B

NumericPoint2D{Int32}(2, 4)

**Fields of composite types are immutable by default**

In [22]:
A.x = 2

LoadError: setfield!: immutable struct of type NumericPoint2D cannot be changed

## Mutable composite types

In [23]:
mutable struct MutablePoint2D{T<:Real}
    x::T
    y::T
end 

A = MutablePoint2D{Int32}(1, 2)
A.x = 2
@show A.x;

A.x = 2


## Operations on types

In [24]:
isa(1, Int)

true

In [25]:
typeof(1)

Int64

In [26]:
supertype(UInt32)

Unsigned

## Constructors

In [27]:
struct Polar{T<:Real}
    r::T
    θ::T
end

A = Polar(1.0, π / 4)
A.θ

0.7853981633974483

In [28]:
function Polar(A::NumericPoint2D{T}) where T
    r = sqrt(A.x ^ 2 + A.y ^ 2)
    θ = atan(A.y, A.x)
    Polar{T}(r, θ)
end

Polar

In [29]:
A = NumericPoint2D{Float32}(cos(π / 4), sin(π / 4))
B = Polar(A)
@show B.r
@show B.θ
typeof(B.r)

B.r = 0.99999994f0
B.θ = 0.7853982f0


Float32

In [30]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
function hideElements(elements, start) {
for(var i = 0, length = elements.length; i < length;i++) {
    if(i >= start) {
        elements[i].style.display = "none";
    }
}
}
var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)
"""